# Text-to-Image AI Service in Google Colab

This notebook sets up and runs a text-to-image generation service using Stable Diffusion with TensorFlow and KerasCV.

## Requirements
- Enable GPU runtime: Runtime → Change runtime type → Hardware accelerator → GPU
- High-RAM runtime recommended for better performance

## Features
- 🎨 Stable Diffusion image generation
- 🚀 FastAPI REST API
- 🌐 Public URL via ngrok
- 📱 Interactive web interface

## Step 1: Check GPU and Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

# Check Python version
import sys
print(f"Python version: {sys.version}")

# Set up environment variables
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['COLAB_GPU'] = '1'

print("✅ Environment check complete")

## Step 2: Install Dependencies

In [ ]:
# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libgl1-mesa-glx libglib2.0-0 libsm6 libxext6 libxrender-dev libgomp1

print("✅ System dependencies installed")

In [ ]:
# Install Python dependencies
!pip install -q tensorflow>=2.15.0
!pip install -q keras-cv>=0.6.0
!pip install -q fastapi>=0.104.0
!pip install -q uvicorn[standard]>=0.24.0
!pip install -q python-multipart>=0.0.6
!pip install -q pillow>=10.0.0
!pip install -q pydantic>=2.5.0
!pip install -q pydantic-settings>=2.1.0
!pip install -q python-dotenv>=1.0.0
!pip install -q structlog>=23.2.0
!pip install -q nest-asyncio
!pip install -q pyngrok

print("✅ Python dependencies installed")

## Step 3: Setup ngrok Authentication (REQUIRED)

**IMPORTANT**: ngrok now requires authentication even for free usage.

1. **Sign up for free**: Go to https://dashboard.ngrok.com/signup
2. **Get your auth token**: Visit https://dashboard.ngrok.com/get-started/your-authtoken  
3. **Copy the token**: It looks like `2abc123_def456ghi789jkl...`
4. **Replace the token below**: Change `YOUR_NGROK_AUTH_TOKEN_HERE` to your actual token

In [ ]:
# Set up ngrok authentication (REQUIRED)
# Get your auth token from: https://dashboard.ngrok.com/get-started/your-authtoken

from pyngrok import ngrok, conf

# REPLACE THIS with your actual ngrok auth token:
NGROK_AUTH_TOKEN = "YOUR_NGROK_AUTH_TOKEN_HERE"

if NGROK_AUTH_TOKEN == "YOUR_NGROK_AUTH_TOKEN_HERE":
    print("❌ ERROR: Please set your ngrok auth token!")
    print("1. Go to: https://dashboard.ngrok.com/signup (free signup)")
    print("2. Get token: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("3. Replace YOUR_NGROK_AUTH_TOKEN_HERE above with your token")
    raise ValueError("ngrok auth token required")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("✅ ngrok authentication configured successfully!")

## Step 4: Create and Run the Text-to-Image Service

In [ ]:
import os
import time
import asyncio
import nest_asyncio
from typing import Optional, List
import tensorflow as tf
import keras_cv
import numpy as np
from PIL import Image
import io
import base64
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
import uvicorn
from pyngrok import ngrok
import threading

# Enable nested asyncio for Colab
nest_asyncio.apply()

print("✅ Imports complete")

In [ ]:
# Configure TensorFlow for Colab
def setup_tensorflow():
    """Configure TensorFlow for Colab environment."""
    gpus = tf.config.experimental.list_physical_devices("GPU")
    if gpus:
        try:
            for gpu in gpus:
                tf.config.experimental.set_memory_growth(gpu, True)
            tf.keras.mixed_precision.set_global_policy("mixed_float16")
            print(f"✅ Configured {len(gpus)} GPU(s) with mixed precision")
        except RuntimeError as e:
            print(f"⚠️ GPU setup error: {e}")
    else:
        print("⚠️ No GPU detected, using CPU (will be slow)")
    return len(gpus) > 0

gpu_available = setup_tensorflow()

In [ ]:
# Define API models
class ImageRequest(BaseModel):
    prompt: str = Field(..., min_length=1, max_length=500, description="Text description of the image")
    num_steps: int = Field(25, ge=10, le=50, description="Number of diffusion steps (lower = faster)")
    guidance_scale: float = Field(7.5, ge=1.0, le=15.0, description="How closely to follow the prompt")
    seed: Optional[int] = Field(None, ge=0, description="Random seed for reproducible results")

class ImageResponse(BaseModel):
    image_base64: str
    prompt: str
    generation_time: float
    parameters: dict

print("✅ API models defined")

In [ ]:
# Create the Stable Diffusion model class
class ColabStableDiffusion:
    """Simplified Stable Diffusion model for Colab."""

    def __init__(self):
        self.model = None
        self.load_model()

    def load_model(self):
        """Load Stable Diffusion model."""
        print("🔄 Loading Stable Diffusion model (this may take a few minutes)...")

        try:
            self.model = keras_cv.models.StableDiffusion(
                img_width=512,
                img_height=512,
                jit_compile=False,  # Disable for Colab compatibility
            )
            print("✅ Stable Diffusion model loaded successfully!")
        except Exception as e:
            print(f"❌ Model loading failed: {e}")
            raise

    def generate_image(self, prompt: str, num_steps: int = 25,
                      guidance_scale: float = 7.5, seed: Optional[int] = None):
        """Generate image from text prompt."""
        if self.model is None:
            raise RuntimeError("Model not loaded")

        if seed is not None:
            tf.random.set_seed(seed)
            np.random.seed(seed)

        try:
            print(f"🎨 Generating image for: {prompt[:50]}...")
            start_time = time.time()

            # Check KerasCV version and use appropriate parameters
            try:
                # Try with guidance_scale first (newer versions)
                generated_images = self.model.text_to_image(
                    prompt=prompt,
                    batch_size=1,
                    num_steps=num_steps,
                    guidance_scale=guidance_scale,
                )
            except TypeError as e:
                if "guidance_scale" in str(e):
                    print("⚠️ Using unconditional_guidance_scale parameter for older KerasCV version")
                    # Fallback for older versions that use unconditional_guidance_scale
                    try:
                        generated_images = self.model.text_to_image(
                            prompt=prompt,
                            batch_size=1,
                            num_steps=num_steps,
                            unconditional_guidance_scale=guidance_scale,
                        )
                    except TypeError:
                        # If still failing, try without guidance parameter
                        print("⚠️ Using basic parameters without guidance scale")
                        generated_images = self.model.text_to_image(
                            prompt=prompt,
                            batch_size=1,
                            num_steps=num_steps,
                        )
                else:
                    raise e

            # Convert to PIL Image with proper normalization
            img_array = generated_images[0]

            # Debug: Check the actual range of values
            print(f"Image array range: {img_array.min():.3f} to {img_array.max():.3f}")

            # Proper normalization for KerasCV output (usually [0,1] range)
            if img_array.max() <= 1.0 and img_array.min() >= 0.0:
                # Standard [0,1] to [0,255] conversion
                img_array = img_array * 255.0
                print("✅ Applied [0,1] → [0,255] normalization")
            elif img_array.max() <= 1.0 and img_array.min() >= -1.0:
                # [-1,1] to [0,255] conversion
                img_array = (img_array + 1.0) * 127.5
                print("✅ Applied [-1,1] → [0,255] normalization")
            else:
                # Already in [0,255] range or other range - normalize to [0,255]
                img_array = ((img_array - img_array.min()) / (img_array.max() - img_array.min())) * 255.0
                print("✅ Applied custom normalization to [0,255]")

            img_array = np.clip(img_array, 0, 255).astype(np.uint8)
            pil_image = Image.fromarray(img_array)

            generation_time = time.time() - start_time
            print(f"✅ Image generated in {generation_time:.1f} seconds")

            return pil_image

        except Exception as e:
            print(f"❌ Generation failed: {e}")
            raise

# Initialize the model (this will take a few minutes)
print("Initializing Stable Diffusion model...")
sd_model = ColabStableDiffusion()

In [ ]:
# Create FastAPI application with web interface
from fastapi.staticfiles import StaticFiles
from fastapi.responses import RedirectResponse

app = FastAPI(
    title="Text-to-Image AI (Colab)",
    description="Stable Diffusion text-to-image generation running in Google Colab",
    version="1.0.0"
)

# Create web interface HTML content
web_interface_html = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Text-to-Image AI Generator</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body { font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); min-height: 100vh; padding: 20px; }
        .container { max-width: 800px; margin: 0 auto; background: rgba(255, 255, 255, 0.95); border-radius: 20px; padding: 30px; box-shadow: 0 20px 40px rgba(0, 0, 0, 0.1); }
        .header { text-align: center; margin-bottom: 30px; }
        .header h1 { color: #333; font-size: 2.5em; margin-bottom: 10px; background: linear-gradient(45deg, #667eea, #764ba2); -webkit-background-clip: text; -webkit-text-fill-color: transparent; background-clip: text; }
        .form-group { margin-bottom: 20px; }
        .form-group label { display: block; margin-bottom: 8px; color: #333; font-weight: 600; }
        .form-group input, .form-group textarea { width: 100%; padding: 12px; border: 2px solid #ddd; border-radius: 8px; font-size: 14px; }
        .form-group textarea { min-height: 80px; resize: vertical; }
        .generate-btn { width: 100%; background: linear-gradient(45deg, #667eea, #764ba2); color: white; border: none; padding: 15px; border-radius: 10px; font-size: 16px; font-weight: 600; cursor: pointer; }
        .generate-btn:disabled { opacity: 0.6; cursor: not-allowed; }
        .loading { display: none; text-align: center; margin: 20px 0; }
        .loading-spinner { border: 4px solid #f3f3f3; border-top: 4px solid #667eea; border-radius: 50%; width: 40px; height: 40px; animation: spin 1s linear infinite; margin: 0 auto 20px; }
        @keyframes spin { 0% { transform: rotate(0deg); } 100% { transform: rotate(360deg); } }
        .result-image { max-width: 100%; border-radius: 15px; box-shadow: 0 10px 30px rgba(0, 0, 0, 0.2); margin: 20px 0; }
        .error-message { background: #f8d7da; color: #721c24; padding: 15px; border-radius: 10px; margin: 20px 0; }
        .success-message { background: #d4edda; color: #155724; padding: 15px; border-radius: 10px; margin: 20px 0; }
        .range-group { display: flex; align-items: center; gap: 10px; }
        .range-value { background: #667eea; color: white; padding: 4px 8px; border-radius: 4px; font-size: 12px; min-width: 35px; text-align: center; }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🎨 Text-to-Image AI</h1>
            <p>Create amazing images with AI-powered Stable Diffusion</p>
        </div>

        <form id="generateForm">
            <div class="form-group">
                <label for="prompt">🖋️ Describe your image:</label>
                <textarea id="prompt" name="prompt" placeholder="a beautiful sunset over mountains, serene lake, photorealistic..." required></textarea>
            </div>

            <div class="form-group">
                <label for="negative_prompt">🚫 What to avoid (optional):</label>
                <input type="text" id="negative_prompt" name="negative_prompt" placeholder="blurry, low quality, people...">
            </div>

            <div class="form-group">
                <label for="num_steps">🔄 Steps: <span class="range-value" id="steps-value">25</span></label>
                <div class="range-group">
                    <input type="range" id="num_steps" name="num_steps" min="10" max="50" value="25" oninput="document.getElementById('steps-value').textContent = this.value">
                </div>
            </div>

            <div class="form-group">
                <label for="guidance_scale">🎯 Prompt Adherence: <span class="range-value" id="guidance-value">7.5</span></label>
                <div class="range-group">
                    <input type="range" id="guidance_scale" name="guidance_scale" min="1" max="15" step="0.5" value="7.5" oninput="document.getElementById('guidance-value').textContent = this.value">
                </div>
            </div>

            <button type="submit" class="generate-btn" id="generateBtn">🚀 Generate Image</button>
        </form>

        <div class="loading" id="loading">
            <div class="loading-spinner"></div>
            <p>🎨 Creating your masterpiece...</p>
        </div>

        <div id="results"></div>
    </div>

    <script>
        const API_BASE_URL = window.location.origin;

        document.getElementById('generateForm').addEventListener('submit', async function(e) {
            e.preventDefault();

            const generateBtn = document.getElementById('generateBtn');
            const loading = document.getElementById('loading');
            const results = document.getElementById('results');

            const formData = new FormData(e.target);
            const requestData = {
                prompt: formData.get('prompt'),
                num_steps: parseInt(formData.get('num_steps')),
                guidance_scale: parseFloat(formData.get('guidance_scale'))
            };

            if (formData.get('negative_prompt')) {
                requestData.negative_prompt = formData.get('negative_prompt');
            }

            generateBtn.disabled = true;
            generateBtn.textContent = '🎨 Generating...';
            loading.style.display = 'block';
            results.innerHTML = '';

            try {
                const response = await fetch(`${API_BASE_URL}/generate`, {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify(requestData)
                });

                if (!response.ok) {
                    const error = await response.json();
                    throw new Error(error.detail || `HTTP ${response.status}`);
                }

                const result = await response.json();
                
                results.innerHTML = `
                    <div class="success-message">
                        <h3>✅ Image Generated Successfully!</h3>
                        <p>Generation time: ${result.generation_time.toFixed(1)} seconds</p>
                    </div>
                    <img src="data:image/png;base64,${result.image_base64}" alt="Generated Image" class="result-image">
                `;

            } catch (error) {
                results.innerHTML = `
                    <div class="error-message">
                        <h3>❌ Generation Failed</h3>
                        <p><strong>Error:</strong> ${error.message}</p>
                    </div>
                `;
            } finally {
                generateBtn.disabled = false;
                generateBtn.textContent = '🚀 Generate Image';
                loading.style.display = 'none';
            }
        });
    </script>
</body>
</html>'''

# Store the HTML for serving
import os
os.makedirs('/tmp/static', exist_ok=True)
with open('/tmp/static/index.html', 'w') as f:
    f.write(web_interface_html)

# Mount static files
app.mount("/static", StaticFiles(directory="/tmp/static"), name="static")

@app.post("/generate", response_model=ImageResponse)
async def generate_image_endpoint(request: ImageRequest):
    """Generate image from text prompt."""
    start_time = time.time()

    try:
        # Run generation in thread pool to avoid blocking
        loop = asyncio.get_event_loop()
        pil_image = await loop.run_in_executor(
            None,
            sd_model.generate_image,
            request.prompt,
            request.num_steps,
            request.guidance_scale,
            request.seed,
        )

        # Convert to base64
        buffer = io.BytesIO()
        pil_image.save(buffer, format="PNG")
        img_base64 = base64.b64encode(buffer.getvalue()).decode()

        generation_time = time.time() - start_time

        return ImageResponse(
            image_base64=img_base64,
            prompt=request.prompt,
            generation_time=generation_time,
            parameters={
                "num_steps": request.num_steps,
                "guidance_scale": request.guidance_scale,
                "seed": request.seed
            }
        )

    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Generation failed: {str(e)}")

@app.get("/health")
async def health_check():
    """Health check endpoint."""
    return {
        "status": "healthy",
        "model_loaded": sd_model.model is not None,
        "gpu_available": gpu_available,
        "environment": "Google Colab"
    }

@app.get("/")
async def root():
    """Root endpoint - redirect to web interface."""
    return RedirectResponse(url="/static/index.html")

print("✅ FastAPI application with web interface created")

## Step 5: Start the Server with Public URL

In [ ]:
# Start the server with ngrok tunnel
import threading
import time
import asyncio
from uvicorn import Config, Server

def start_server():
    """Start the FastAPI server with proper async handling."""
    print("🔄 Starting FastAPI server...")
    try:
        # Create a new event loop for this thread
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        
        # Configure and start the server
        config = Config(app, host="0.0.0.0", port=8000, log_level="info")
        server = Server(config)
        
        # Run the server
        loop.run_until_complete(server.serve())
    except Exception as e:
        print(f"❌ Server startup failed: {e}")

# Start the server in a separate thread BEFORE creating ngrok tunnel
print("🚀 Starting the FastAPI server...")
server_thread = threading.Thread(target=start_server)
server_thread.daemon = True
server_thread.start()

# Wait for server to start
print("⏳ Waiting for server to initialize...")
time.sleep(15)  # Increased wait time

# Test if server is running locally
import requests
server_ready = False
for attempt in range(5):  # Try 5 times
    try:
        test_response = requests.get("http://localhost:8000/health", timeout=5)
        if test_response.status_code == 200:
            print("✅ Server is running locally!")
            server_ready = True
            break
        else:
            print(f"⚠️ Server responded with status: {test_response.status_code}")
    except Exception as e:
        print(f"🔄 Attempt {attempt + 1}/5: Server not ready yet...")
        time.sleep(5)

if not server_ready:
    print("❌ Server failed to start properly. Please restart runtime and try again.")
else:
    # Now start ngrok tunnel (uses the auth token set earlier)
    print("🌐 Creating ngrok tunnel...")
    try:
        public_tunnel = ngrok.connect(8000)
        public_url = str(public_tunnel)  # Convert to string URL
        
        print("\n" + "="*60)
        print("🚀 TEXT-TO-IMAGE AI SERVICE IS LIVE!")
        print("="*60)
        print(f"🌐 Web Interface: {public_url}")
        print(f"🎨 Direct Access: {public_url}/static/index.html")
        print(f"📖 API Documentation: {public_url}/docs")
        print(f"❤️ Health Check: {public_url}/health")
        print(f"🔧 API Generate: {public_url}/generate")
        print("="*60)
        print("\n🎯 HOW TO USE:")
        print("1. 🖱️  Click the Web Interface link above")
        print("2. 🖋️  Enter your image description")
        print("3. ⚙️  Adjust settings as needed")
        print("4. 🚀 Click 'Generate Image'")
        print("5. ⏳ Wait 30-120 seconds for your AI masterpiece!")
        print("\n💡 TIPS:")
        print("• Be descriptive in your prompts")
        print("• Use negative prompts to avoid unwanted elements")
        print("• Lower steps = faster generation")
        print("• Higher guidance = follows prompt more closely")
        print("\n⚠️ Note: Keep this cell running to maintain the service")
        print("="*60)
        
    except Exception as e:
        print(f"❌ Failed to create ngrok tunnel: {e}")

if server_ready:
    print("\n✅ Setup complete! Click the Web Interface link above to start creating images!")
else:
    print("\n❌ Setup failed. Please restart runtime and try again.")

## Step 6: Troubleshooting & Server Check

If you're getting ngrok connection errors, run this cell to diagnose the issue:

In [ ]:
# Troubleshooting: Check if server is running properly
import requests
import time
import subprocess
import threading
import asyncio
from uvicorn import Config, Server

print("🔍 Diagnosing server status...")

# Check if port 8000 is in use
try:
    result = subprocess.run(['netstat', '-tuln'], capture_output=True, text=True)
    if ':8000' in result.stdout:
        print("✅ Port 8000 is in use (server might be running)")
    else:
        print("❌ Port 8000 is not in use (server not running)")
except:
    print("⚠️ Could not check port status")

# Test local server connection
print("\n🔍 Testing local server connection...")
try:
    response = requests.get("http://localhost:8000/health", timeout=10)
    print(f"✅ Local server is responding! Status: {response.status_code}")
    print(f"Response: {response.json()}")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to local server on port 8000")
    print("🔄 Trying to restart the server...")
    
    # Try to restart server with proper async handling
    def restart_server():
        try:
            # Create a new event loop for this thread
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)
            
            # Configure and start the server
            config = Config(app, host="0.0.0.0", port=8000, log_level="info")
            server = Server(config)
            
            # Run the server
            loop.run_until_complete(server.serve())
        except Exception as e:
            print(f"❌ Server restart failed: {e}")
    
    server_thread = threading.Thread(target=restart_server)
    server_thread.daemon = True
    server_thread.start()
    
    print("⏳ Waiting 20 seconds for server restart...")
    time.sleep(20)
    
    # Test again
    for attempt in range(3):
        try:
            response = requests.get("http://localhost:8000/health", timeout=5)
            print(f"✅ Server restarted successfully! Status: {response.status_code}")
            break
        except:
            print(f"🔄 Restart attempt {attempt + 1}/3...")
            time.sleep(5)
    else:
        print("❌ Server restart failed")

except Exception as e:
    print(f"❌ Error testing server: {e}")

# Test ngrok tunnel if it exists
if 'public_url' in globals():
    print(f"\n🔍 Testing ngrok tunnel: {public_url}")
    try:
        response = requests.get(f"{public_url}/health", timeout=30)
        print(f"✅ Ngrok tunnel is working! Status: {response.status_code}")
    except Exception as e:
        print(f"❌ Ngrok tunnel test failed: {e}")
        print("💡 Try running the server setup cell again")
else:
    print("\n⚠️ No ngrok tunnel found. Run the server setup cell first.")

print("\n" + "="*50)
print("💡 TROUBLESHOOTING TIPS:")
print("="*50)
print("1. If local server fails: Restart runtime and run all cells again")
print("2. If ngrok fails: Check your auth token is set correctly")  
print("3. If still failing: Try running cells one by one with delays")
print("4. For memory issues: Enable High-RAM runtime")
print("5. For async errors: Make sure nest_asyncio is installed")
print("="*50)

## Step 7: Test the API (Optional)

In [ ]:
# Test the API directly from the notebook
import requests
import json
from IPython.display import Image as IPImage
from io import BytesIO

def test_generation(prompt, num_steps=25):
    """Test image generation directly."""
    print(f"Testing generation with prompt: {prompt}")
    
    # Check if we have the public URL
    if 'public_url' not in globals():
        print("❌ Error: No public URL found. Run the server setup cell first.")
        return None
    
    # Ensure we have a proper URL string
    test_url = public_url if isinstance(public_url, str) else str(public_url)
    if not test_url.startswith(('http://', 'https://')):
        print("❌ Error: Invalid URL format")
        return None

    try:
        print(f"🌐 Making request to: {test_url}")
        
        # Make API request
        response = requests.post(
            f"{test_url}/generate",
            json={
                "prompt": prompt,
                "num_steps": num_steps,
                "guidance_scale": 7.5
            },
            timeout=300  # 5 minute timeout for generation
        )

        if response.status_code == 200:
            result = response.json()
            print(f"✅ Generation completed in {result['generation_time']:.1f} seconds")

            # Decode and display image
            img_data = base64.b64decode(result['image_base64'])
            return IPImage(img_data)
        else:
            print(f"❌ Error: {response.status_code} - {response.text}")
            return None
            
    except requests.exceptions.RequestException as e:
        print(f"❌ Request failed: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error: {e}")
        return None

# Wait a moment for everything to be ready
print("⏳ Waiting for services to be ready...")
time.sleep(5)

# Test with a simple prompt
print("🎨 Testing image generation...")
test_image = test_generation("a cute cat sitting in a garden", num_steps=20)
if test_image:
    display(test_image)
    print("🎉 Success! Your text-to-image service is working!")
else:
    print("🔄 If the test failed, try running the troubleshooting cell above.")
    print("💡 Common solutions:")
    print("   - Wait a bit longer (model might still be loading)")
    print("   - Run the troubleshooting cell to check server status")
    print("   - Restart runtime and run all cells again")

## Step 8: Keep the Service Running